In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import jieba

In [2]:
path = 'data/poetry'

In [3]:
def load_data(path):
    file = path
    datasets = pd.read_csv(file, header=0)
    cats = datasets.cat
    labels = datasets.label
    reviews = datasets.review
    rows = np.random.randint(0, len(datasets), len(datasets))
    return np.array(cats[rows]), np.array(reviews[rows]), np.array(labels[rows])

In [4]:
f = open(path)
a = f.read().split('\n')

In [5]:
print (a)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
a[0]

'首春:寒随穷律变，春逐鸟声开。初风飘带柳，晚雪间花梅。碧林青旧竹，绿沼翠新苔。芝田初雁去，绮树巧莺来。'

In [7]:
aa = a[0]

In [8]:
pun = [':', '，', '。', '!']
def cutreviews(texts):
    sent_words = []
    for i in range(len(texts)):
        sent_word = list(jieba.lcut(texts[i]))
#         print (sent_word)
        _sent_words = []
        for x in sent_word:
            if x not in pun:
                _sent_words += x
        
        _sent_word = ''.join(str(x) for x in _sent_words)
        sent_words.append(_sent_word)
    return sent_words  

In [9]:
datasets = cutreviews(a)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/gj/k8xyg57d6ggdm6g6l0d0mmzh0000gn/T/jieba.cache
Loading model cost 0.812 seconds.
Prefix dict has been built succesfully.


In [10]:
print (type(datasets))

<class 'list'>


In [11]:
datasets[0]

'首春寒随穷律变春逐鸟声开初风飘带柳晚雪间花梅碧林青旧竹绿沼翠新苔芝田初雁去绮树巧莺来'

## Create Model

In [12]:
import torch
import torch.nn as nn

In [39]:
class PoetryModel(nn.Module):
    def __init__(self, vocab, embedding_dim, hidden_dim):
        super(PoetryModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(len(vocab), embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers=2)
        self.linear = nn.Linear(self.hidden_dim, len(vocab))
        
    def forward(self, input, hidden):
        seq_len, batch_size = input.size()
        embeds = self.embeddings(input)
        output, hidden = self.lstm(embeds, hidden)
        output = self.linear(output.view(seq_len * batch_size, -1))
        return output. hidden
        
    def initHidden(self):
        return torch.zaros(1, self.hidden_dim)

## Preparing for training

In [14]:
import numpy as np

In [15]:
def randomChoice(data):
    if isinstance(data, list):
        data = np.array(data)
    rows = np.random.randint(0, len(data), len(data))
    return data[rows]

In [16]:
def randomchoiceone(data):
    data = np.array(data)
    return data[np.random.randint(0, len(data)-1)]

In [18]:
ran_datasets = randomChoice(datasets)

In [19]:
ran_datasets[1]

'酬滁州李十六使君见赠满镜悲华发空山寄此身白云家自有黄卷业长贫懒任垂竿老狂因酿黍春桃花迷圣代桂树狎幽人幢盖方临郡柴荆忝作邻但愁千骑至石路却生尘'

In [20]:
ran_dataset = randomchoiceone(datasets)

In [21]:
ran_dataset

'横吹曲辞关山月凄凄还切切戍客多离别何处最伤心关山见秋月关月竟如何由来远近过始经玄兔塞终绕白狼河忽忆秦楼妇流光应共有已得并蛾眉还知揽纤手去岁照同行比翼复连形今宵照独立顾影自茕茕馀晖渐西落夜夜看如昨借问映旌旗何如鉴帷幕拂晓朔风悲蓬惊雁不飞几时征戍罢还向月中归'

In [22]:
datatest = ran_datasets[:2]

In [23]:
datatest[0]

'相如琴台闻有雍容地千年无四邻园院风烟古池台松槚春云疑作赋客月似听琴人寂寂啼莺处空伤游子神'

## Make a vocabulary

In [24]:
def vocabb(data: np.ndarray):
    vocabdict = []
    for i in range(len(data)):
        row = data[i]
        for word in data[i]:
            if word not in vocabdict:
                vocabdict.append(word)
    num = range(0, len(vocabdict)-1)
    
    return dict(zip(vocabdict, num))

In [25]:
vocabl = vocabb(ran_datasets)

In [26]:
def vocabre(vocab: dict):
    return dict((v,k) for k, v in vocab.items())

In [27]:
print (len(vocabl))

7110


In [28]:
vocabl["和"]

278

In [29]:
re_vocab = vocabre(vocabl)

In [30]:
re_vocab[283]

'众'

## One-hot vector for category

In [31]:
def chartoindex(char, vocab):
    num = vocab[char]
    return num

In [32]:
def chartotensor(char, vocab):
    tensor = torch.zeros(1, len(vocab))
    tensor[0][chartoindex(char, vocab)] = 1
    return tensor

In [33]:
def inputtensor(line, vocab):
    tensor = torch.zeros(len(line), 1, len(vocab))
    for li in range(len(line)):
        char = line[li]
        tensor[li][0][vocab[char]] = 1
        return tensor

In [34]:
def targettensor(line, vocab):
    char_indexes = [vocab[line[li]] for li in range(1, len(line))]
    char_indexes.append(len(line))
    return torch.LongTensor(char_indexes)

In [35]:
def randomtraingingexample(data, vocab):
    line = randomchoiceone(data)
    input_line_tensor = inputtensor(line, vocab)
    target_line_tensor = targettensor(line, vocab)
    return input_line_tensor, target_line_tensor

In [36]:
randomtraingingexample(datasets, vocabl)

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]]]),
 tensor([ 337, 1326,  505,  182, 2063,   66,  702,  816,  104, 1425, 1769,  500,
           56,  193,  349, 1192,   11,  971, 3367,  599,  396,  626,   66, 2274,
         1108, 2944,  903,  475,  205, 1184,  948, 1499,  448, 3182,  520, 1397,
          958,  163,  513,  242,  242, 2705,   17,  680, 1185, 1185, 1643,  795,
          162, 2063,  101,  111,  350, 2080,  337, 1317,  583,  198, 2978,  139,
         1060, 1735,  112, 3111,  126, 1152,  277,  140,  702,   70]))

In [37]:
print (inputtensor(datatest[1], vocabl).shape)

torch.Size([70, 1, 7110])


## Train

In [40]:
#rnn = RNN(len(vocabl), 1000, len(vocabl)).cuda()
rnn = PoetryModel(vocabl, 256, 500)
# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001

optimizer = torch.optim.Adam(rnn.parameters(), learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)

In [41]:
def train(input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden().cuda()
    #rnn.zero_grad()
    loss = 0
    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(input_line_tensor[i], hidden)
        optimizer.zero_grad()
        l = criterion(output, target_line_tensor[i])
        loss += l
        optimizer.step()
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
        
    return output, loss.item() / input_line_tensor.size(0)

In [42]:
n_iters = 10000
print_every = 200
plot_every = 500
all_losses = []
total_loss = 0

for iter in range(0, n_iters+1):
    input_line_tensor, target_line_tensor = randomtraingingexample(ran_datasets, vocabl)
    input_line_tensor = input_line_tensor.cuda()
    target_line_tensor = target_line_tensor.cuda()
#     print (input_line_tensor)
#     print (target_line_tensor)
    output, loss = train(input_line_tensor, target_line_tensor)
    total_loss += loss
    
    if iter % print_every == 0:
        print('(%d %d%%) %.4f' % (iter, iter / n_iters * 100, loss))
        
    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
max_lenght = 7

In [ ]:
def sample(vocab, revocab, start_char='成' ):
    with torch.no_grad():
        input = inputtensor(start_char, vocab)
        input = input.cuda()
        hidden = rnn.initHidden().cuda()
        output_p = start_char
        
        for i in range(max_lenght):
            output, hidden = rnn(input[0], hidden)
            topv, topi = output.topk(1)
            topi = int(topi[0][0])
#            print (topi)
            if topi == len(vocab):
                break
            else:
                char = revocab[topi]
                output_p += char
    return output_p 

In [ ]:
def samples(vocab,revocab, start_chars='天上'):
    for start_char in start_chars:
        print (sample(vocab, revocab, start_char))

In [ ]:
samples(vocabl,re_vocab, start_chars='天上人间')